In [1]:
import numpy as np
import pandas as pd
import requests
import json
from datetime import datetime
from datetime import timedelta
# import everything from tkinter module
from tkinter import *

# import messagebox from tkinter module
import tkinter.messagebox
import time


In [2]:
# Create a messagebox showinfo
def onClick():
    tkinter.messagebox.showinfo("Welcome to GFG.", "Hi I'm your message")
    
#Create alert function
def alert ():
        #Creat alert
    root = tkinter.Tk()

    # root window title and dimension
    root.title("New opportunity")
    root.geometry('500x300') 

    # Create a messagebox showinfo
    #def onClick():
        #tkinter.messagebox.showinfo("Welcome to GFG.", "Hi I'm your message")

    # Create a Button
    button = Button(root, text="I understand", command=onClick, height=5, width=10)

    # Set the position of button on the top of window.
    button.pack(side='bottom')
    root.mainloop()
    

#Set ophash inadvance
ophash = []

#arbitrage bot contract


In [3]:

for t in range(1):
    #times = (datetime.now()- timedelta(days=1)).isoformat(timespec='seconds')
    #query = ' query MyQuery {\n  event(\n    limit: 200\n        where: {marketplace_event_type: {_in: ["list_create", "offer_create", "offer_floor_create"]}, timestamp: {_gt: "2023-01-17T20:17:43"}, token: {highest_offer: {_is_null: false}, lowest_ask: {_is_null: false}, last_metadata_update: {_lt: "'+ f'{times}' + '"}}}\n\n    order_by: {timestamp: desc, token_pk: asc}\n  ) {\n    timestamp\n    marketplace_event_type\n      creator_address\n    token_pk\n    price\n    token {\n      fa_contract\n      highest_offer\n      lowest_ask\n      token_id\n      royalties {\n        decimals\n        amount\n      }\n    }\n    ophash\n  }\n}\n\n '
    query = """ 
        query MyQuery {
        event(
            limit: 200
                where: {marketplace_event_type: {_in: ["list_create", "offer_create", "offer_floor_create"]}, timestamp: {_gt: "2023-01-17T20:17:43"}, token: {highest_offer: {_is_null: false}, lowest_ask: {_is_null: false}}}

            order_by: {timestamp: desc, token_pk: asc}
        ) {
            timestamp
            marketplace_event_type
            creator_address
            token_pk
            price
            token {
            fa_contract
            highest_offer
            lowest_ask
            token_id
            royalties {
                decimals
                amount
            }
            }
            ophash
        }
        }
        """

    url = "https://data.objkt.com/v3/graphql"
    r = requests.post(url, json={'query': query})
    #print(r.status_code)
    json_data = json.loads(r.text)
    df = pd.json_normalize(json_data, record_path=['data','event']) # this parameter adjust to new queries
    df['price'] =df['price']/1000000
    df['token.highest_offer'] =df['token.highest_offer']/1000000 #Adjust the price, exclude the decimal
    df['token.lowest_ask'] =df['token.lowest_ask']/1000000
    #Counter the different offer
    #a = pd.DataFrame.from_dict(Counter(df['token_pk']), orient= 'index')
    #df.loc[[i in a[a[0]>1].index for i in df['token_pk']]] #A boolean column as criteria for selecting the record with token_pk appears multiple times
    royalties = pd.json_normalize(json_data, record_path=['data', 'event', 'token', 'royalties'], meta=[['data','event','ophash']]) # this parameter adjust to new queries

    b = [] #Loop from df['ophash']
    for i in df['ophash']:
        a = royalties[royalties['data.event.ophash']== i ]
        b.extend([sum(a.amount*(1/10**a.decimals))]) #Operate of pandas series
    df['royalties'] = b
        

    df_chance = df[
        ((df['token.highest_offer']*(1-df['royalties'])>df['price']) 
         & (df['marketplace_event_type'] == 'list_create'))
        | ((df['price']*(1-df['royalties'])>df['token.lowest_ask']) 
           & (df['marketplace_event_type'] != 'list_create'))
        ] #filter the matrix first
    df_chance = df_chance[[i not in ophash for i in df_chance['ophash']]] #filter the ophash collected before
    #df_chance = df_chance[[i not in ['KT1J6GLMjrWgQ73knp2GZKUznTyc3b2Ek1mF'] for i in df_chance['token.fa_contract']]] #filter the ophash collected before
        
    if (len(df_chance)> 0):
        ophash.extend(df_chance['ophash'])
        print(df_chance[['timestamp', 'price', 'token.fa_contract','token.highest_offer','token.lowest_ask','token.token_id','royalties', 'ophash','creator_address', 'marketplace_event_type']])
        
        df_list_all= pd.Series([])
        df_offer_all = pd.Series([])
        for j in range(len(df_chance)):
            k=df_chance.iloc[j]
            if k['marketplace_event_type'] == "list_create":
                query = ' \nquery MyQuery {\n  listing(\n    where: {ophash: {_eq: "'+ f'{k.ophash}' + '"}, token_pk:{_eq: "'+ f'{k.token_pk}' + '"} , status: {_eq: "active"}}\n    limit: 10\n    order_by: {price: asc}\n  ) {\n    id\n    price\n    token_pk\n    seller_address\n    status\n    timestamp\n    bigmap_key\n    ophash\n    fa_contract\n    token {\n      token_id\n    }\n  }\n  offer(\n    limit: 10\n    where: {status: {_eq: "active"}, fa_contract: {_eq: "'+ f'{k["token.fa_contract"]}' + '"}, token: {token_id: {_eq: "'+f'{k["token.token_id"]}'+'"}}}\n  ) {\n    price\n    ophash\n    buyer_address\n    token_pk\n    collection_offer\n    fa_contract\n    status\n    timestamp\n    token {\n      token_id\n    }\n    bigmap_key\n  }\n}\n\n'
                #url = "https://data.objkt.com/v2/graphql"
                r_sub = requests.post(url, json={'query': query})
                #print(r.status_code)
                json_sub = json.loads(r_sub.text)
                df_list = pd.json_normalize(json_sub, record_path=['data','listing'])
                df_offer = pd.json_normalize(json_sub, record_path=['data','offer'])
                df_list_all=pd.concat([df_list, df_list_all])
                df_offer_all = pd.concat([df_offer, df_offer_all])
                
                
            else:
                query = ' \nquery MyQuery {\n  offer(\n    where: {ophash: {_eq: "' + f'{k.ophash}' + '"},token_pk:{_eq: "'+ f'{k.token_pk}' + '"}, status: {_eq: "active"}}\n    limit: 10\n    order_by: {price: desc}\n  ) {\n    id\n    price\n    token_pk\n    buyer_address\n    status\n    timestamp\n    bigmap_key\n    ophash\n    fa_contract\n    token {\n      token_id\n    }\n  }\n  listing(\n    limit: 10\n    where: {status: {_eq: "active"}, fa_contract: {_eq: "' + f'{k["token.fa_contract"]}' + '"}, token: {token_id: {_eq: "'+f'{k["token.token_id"]}'+'"}}}\n    order_by: {price: asc}\n  ) {\n    token_pk\n    price\n    ophash\n    seller_address  \n    fa_contract\n    status\n    timestamp\n    token {\n      token_id\n    }\n    bigmap_key\n  }\n}\n\n'
                r_sub = requests.post(url, json={'query': query})
                json_sub = json.loads(r_sub.text)
                df_list = pd.json_normalize(json_sub, record_path=['data','listing'])
                df_offer = pd.json_normalize(json_sub, record_path=['data','offer'])
                df_list_all=pd.concat([df_list, df_list_all])
                df_offer_all = pd.concat([df_offer, df_offer_all])
                #Sub summary:
            print(df_list_all)
            print(df_offer_all)
            
            #Out of if statement, to get unique value
            if len(df_list_all) !=0 != len(df_offer_all):  #Aviod blank records, the length has to be non-zero
                df_list_all.drop_duplicates(inplace= True,subset=['price']) #This sentence makes sure only 1 records picked up each time, use price as the key
                df_offer_all.drop_duplicates(inplace= True,subset=['price'])
                for l in range(len(df_chance)):
                    index_list = [i == df_chance.iloc[l]['token.lowest_ask']*10**6 for i in df_list_all['price']]
                    index_offer = [i == df_chance.iloc[l]['token.highest_offer']*10**6 for i in df_offer_all['price']]
                    if [(True in index_list) &(True in index_offer)]: #Only 1 true value in each series
                        #Create the Json
                        """ sent = {"fulfill_ask":{"ask_id": int(df_list_all[df_list_all['price'] == df_chance.loc[l]['token.lowest_ask']*10**6]['bigmap_key'])},
                        "fulfill_offer": {"offer_id": int(df_offer_all[index_offer]['bigmap_key']), "token_id": int(df_offer_all[index_offer]['token.token_id'])}} """
                        sent = {"ask_id": int(df_list_all[df_list_all['price'] == df_chance.iloc[l]['token.lowest_ask']*10**6]['bigmap_key']), "offer_id": int(df_offer_all[index_offer]['bigmap_key']), "token_id": int(df_offer_all[index_offer]['token.token_id'])}
                        #Sent the Json, not keeping the data
                        requests.post('http://127.0.0.1:3001/', json=sent)
                    else:
                        print('no matched value')
            else:
                print('missing active records')
        alert()
        
    elif (t%60 == 0):
        print(f'Running at {datetime.now().isoformat(timespec="minutes")}, no opportunity in loop {t}')
        
        
    time.sleep(12)
        
    
    #PermissionError
    #ConnectionError
    #Suspicious contract: KT1J6GLMjrWgQ73knp2GZKUznTyc3b2Ek1mF
    #Suspicious creator: tz1XqrVXcRKDLeWFNWMuDaxhhvYu6zE7WzZc
    

Running at 2023-02-12T09:17, no opportunity in loop 0


In [4]:

for t in range(7200):
    #times = (datetime.now()- timedelta(days=1)).isoformat(timespec='seconds')
    #query = ' query MyQuery {\n  event(\n    limit: 200\n        where: {marketplace_event_type: {_in: ["list_create", "offer_create", "offer_floor_create"]}, timestamp: {_gt: "2023-01-17T20:17:43"}, token: {highest_offer: {_is_null: false}, lowest_ask: {_is_null: false}, last_metadata_update: {_lt: "'+ f'{times}' + '"}}}\n\n    order_by: {timestamp: desc, token_pk: asc}\n  ) {\n    timestamp\n    marketplace_event_type\n      creator_address\n    token_pk\n    price\n    token {\n      fa_contract\n      highest_offer\n      lowest_ask\n      token_id\n      royalties {\n        decimals\n        amount\n      }\n    }\n    ophash\n  }\n}\n\n '
    query = """ 
        query MyQuery {
  event(
    limit: 20
    where: {marketplace_event_type: {_in: ["list_create", "offer_create", "offer_floor_create"]}, timestamp: {_gt: "2023-01-17T20:17:43"}, token: {highest_offer: {_is_null: false}, lowest_ask: {_is_null: false}}, 
    creator_address: {_neq: "tz1XqrVXcRKDLeWFNWMuDaxhhvYu6zE7WzZc"}}
    order_by: {timestamp: desc, token_pk: asc}
  ) {
    timestamp
    marketplace_event_type
    creator_address
    token_pk
    price
    token {
      fa_contract
      highest_offer
      lowest_ask
      token_id
      royalties {
        decimals
        amount
      }
    }
    ophash
  }
}
        """

    url = "https://data.objkt.com/v3/graphql"
    r = requests.post(url, json={'query': query})
    #print(r.status_code)
    json_data = json.loads(r.text)
    df = pd.json_normalize(json_data, record_path=['data','event']) # this parameter adjust to new queries
    df['price'] =df['price']/1000000
    df['token.highest_offer'] =df['token.highest_offer']/1000000 #Adjust the price, exclude the decimal
    df['token.lowest_ask'] =df['token.lowest_ask']/1000000
    #Counter the different offer
    #a = pd.DataFrame.from_dict(Counter(df['token_pk']), orient= 'index')
    #df.loc[[i in a[a[0]>1].index for i in df['token_pk']]] #A boolean column as criteria for selecting the record with token_pk appears multiple times
    royalties = pd.json_normalize(json_data, record_path=['data', 'event', 'token', 'royalties'], meta=[['data','event','ophash']]) # this parameter adjust to new queries

    b = [] #Loop from df['ophash']
    for i in df['ophash']:
        a = royalties[royalties['data.event.ophash']== i ]
        b.extend([sum(a.amount*(1/10**a.decimals))]) #Operate of pandas series
    df['royalties'] = b
        

    df_chance = df[
        ((df['token.highest_offer']*(1-df['royalties'])>df['price']) 
         & (df['marketplace_event_type'] == 'list_create'))
        | ((df['price']*(1-df['royalties'])>df['token.lowest_ask']) 
           & (df['marketplace_event_type'] != 'list_create'))
        ] #filter the matrix first
    df_chance = df_chance[[i not in ophash for i in df_chance['ophash']]] #filter the ophash collected before
    df_chance.drop_duplicates(inplace= True,subset=['token.fa_contract']) #This sentence makes sure only 1 records picked up each time, use price as the key

    #df_chance = df_chance[[i not in ['KT1J6GLMjrWgQ73knp2GZKUznTyc3b2Ek1mF'] for i in df_chance['token.fa_contract']]] #filter the ophash collected before
        
    if (len(df_chance)> 0):
        ophash.extend(df_chance['ophash'])
        print(df_chance[['timestamp', 'price', 'token.fa_contract','token.highest_offer','token.lowest_ask','token.token_id','royalties', 'ophash','creator_address', 'marketplace_event_type']])
        

        for j in range(len(df_chance)):
            df_list_all= pd.Series([]) 
            df_offer_all = pd.Series([])
            k=df_chance.iloc[j]
            if k['marketplace_event_type'] == "list_create":
                query = ' \nquery MyQuery {\n  listing(\n    where: {ophash: {_eq: "'+ f'{k.ophash}' + '"}, token_pk:{_eq: "'+ f'{k.token_pk}' + '"} , status: {_eq: "active"}}\n    limit: 10\n    order_by: {price: asc}\n  ) {\n    id\n    price\n    token_pk\n    seller_address\n    status\n    timestamp\n    bigmap_key\n    ophash\n    fa_contract\n    token {\n      token_id\n    }\n  }\n  offer(\n    limit: 10\n    where: {status: {_eq: "active"}, fa_contract: {_eq: "'+ f'{k["token.fa_contract"]}' + '"}, token: {token_id: {_eq: "'+f'{k["token.token_id"]}'+'"}}}\n  ) {\n    price\n    ophash\n    buyer_address\n    token_pk\n    collection_offer\n    fa_contract\n    status\n    timestamp\n    token {\n      token_id\n    }\n    bigmap_key\n  }\n}\n\n'
                #url = "https://data.objkt.com/v2/graphql"
                r_sub = requests.post(url, json={'query': query})
                #print(r.status_code)
                json_sub = json.loads(r_sub.text)
                df_list = pd.json_normalize(json_sub, record_path=['data','listing'])
                df_offer = pd.json_normalize(json_sub, record_path=['data','offer'])
                df_list_all=pd.concat([df_list, df_list_all])
                df_offer_all = pd.concat([df_offer, df_offer_all])
                
                
            else:
                query = ' \nquery MyQuery {\n  offer(\n    where: {ophash: {_eq: "' + f'{k.ophash}' + '"},token_pk:{_eq: "'+ f'{k.token_pk}' + '"}, status: {_eq: "active"}}\n    limit: 10\n    order_by: {price: desc}\n  ) {\n    id\n    price\n    token_pk\n    buyer_address\n    status\n    timestamp\n    bigmap_key\n    ophash\n    fa_contract\n    token {\n      token_id\n    }\n  }\n  listing(\n    limit: 10\n    where: {status: {_eq: "active"}, fa_contract: {_eq: "' + f'{k["token.fa_contract"]}' + '"}, token: {token_id: {_eq: "'+f'{k["token.token_id"]}'+'"}}}\n    order_by: {price: asc}\n  ) {\n    token_pk\n    price\n    ophash\n    seller_address  \n    fa_contract\n    status\n    timestamp\n    token {\n      token_id\n    }\n    bigmap_key\n  }\n}\n\n'
                r_sub = requests.post(url, json={'query': query})
                json_sub = json.loads(r_sub.text)
                df_list = pd.json_normalize(json_sub, record_path=['data','listing'])
                df_offer = pd.json_normalize(json_sub, record_path=['data','offer'])
                df_list_all=pd.concat([df_list, df_list_all])
                df_offer_all = pd.concat([df_offer, df_offer_all])
                #Sub summary:
            print(df_list_all)
            print(df_offer_all)
            
            #Out of if statement, to get unique value
            if len(df_list_all) !=0 != len(df_offer_all):  #Aviod blank records, the length has to be non-zero
                df_list_all.drop_duplicates(inplace= True,subset=['price']) #This sentence makes sure only 1 records picked up each time, use price as the key
                df_offer_all.drop_duplicates(inplace= True,subset=['price'])
                for l in range(len(df_chance)):
                    index_list = [i == df_chance.iloc[l]['token.lowest_ask']*10**6 or i == df_chance.iloc[l]['price']*10**6 for i in df_list_all['price']]
                    index_offer = [i == df_chance.iloc[l]['token.highest_offer']*10**6 or i == df_chance.iloc[l]['price']*10**6 for i in df_offer_all['price']] #It could be a problem
                    if [(True in index_list) &(True in index_offer)] == [True]: #Only 1 true value in each series
                        #Create the Json
                        """ sent = {"fulfill_ask":{"ask_id": int(df_list_all[df_list_all['price'] == df_chance.loc[l]['token.lowest_ask']*10**6]['bigmap_key'])},
                        "fulfill_offer": {"offer_id": int(df_offer_all[index_offer]['bigmap_key']), "token_id": int(df_offer_all[index_offer]['token.token_id'])}} """
                        sent = {"ask_id": int(df_list_all[df_list_all['price'] == df_chance.iloc[l]['token.lowest_ask']*10**6]['bigmap_key']), "offer_id": int(df_offer_all[index_offer]['bigmap_key']), "token_id": int(df_offer_all[index_offer]['token.token_id'])}
                        #Sent the Json, not keeping the data
                        #requests.post('http://127.0.0.1:3001/', json=sent)
                        print(sent)
                    else:
                        print('no matched value')
            else:
                print('missing active records')
        alert()
        
    elif (t%60 == 0):
        print(f'Running at {datetime.now().isoformat(timespec="minutes")}, no opportunity in loop {t}')
        
        
    time.sleep(12)
        
    
    #PermissionError
    #ConnectionError
    #Suspicious contract: KT1J6GLMjrWgQ73knp2GZKUznTyc3b2Ek1mF
    #Suspicious creator: tz1XqrVXcRKDLeWFNWMuDaxhhvYu6zE7WzZc
    

Running at 2023-02-12T09:17, no opportunity in loop 0
Running at 2023-02-12T09:29, no opportunity in loop 60
Running at 2023-02-12T09:42, no opportunity in loop 120
Running at 2023-02-12T09:54, no opportunity in loop 180
Running at 2023-02-12T10:06, no opportunity in loop 240
Running at 2023-02-12T10:18, no opportunity in loop 300
Running at 2023-02-12T10:31, no opportunity in loop 360
Running at 2023-02-12T10:43, no opportunity in loop 420
Running at 2023-02-12T10:55, no opportunity in loop 480
Running at 2023-02-12T11:07, no opportunity in loop 540
Running at 2023-02-12T11:20, no opportunity in loop 600
Running at 2023-02-12T11:32, no opportunity in loop 660
Running at 2023-02-12T13:18, no opportunity in loop 720
Running at 2023-02-12T13:30, no opportunity in loop 780
Running at 2023-02-12T13:42, no opportunity in loop 840
Running at 2023-02-12T13:55, no opportunity in loop 900
Running at 2023-02-12T14:07, no opportunity in loop 960
Running at 2023-02-12T14:19, no opportunity in loop

In [ ]:
sent

SyntaxError: invalid syntax (3299761268.py, line 1)

In [ ]:
df_list_all

,id,price,token_pk,seller_address,status,timestamp,bigmap_key,ophash,fa_contract,token.token_id,0
0,5683298.0,5000000.0,14036125.0,tz1bkoDd8NAkfuv1oVU4sCY7JiSqA8Qs6jU1,active,2023-02-07T13:32:14+00:00,867963.0,onscpBonkKcPy2fmmDx4ezxRAsCE9QRTHRJoCE6icbLjmx...,KT1KEa8z6vWXDJrVqtMrAeDVzsvxat3kHaCE,350147,NaN


In [ ]:
df_offer_all

,price,ophash,buyer_address,token_pk,collection_offer,fa_contract,status,timestamp,bigmap_key,token.token_id,0
0,6000000.0,op47qw5eu9J3WAeGhX8khBbgACVH7wP6RvwF685SMbss85...,tz1ZQNJ2jb7WcoYjFaSsboAC6YUs1T577A95,14036125.0,None,KT1KEa8z6vWXDJrVqtMrAeDVzsvxat3kHaCE,active,2023-02-07T12:57:44+00:00,53108.0,350147,NaN


In [ ]:
requests.post('http://127.0.0.1:3000/', json=sent)


<Response [200]>

In [ ]:
[(True in index_list) &(True in index_offer)] == [False]

True

In [ ]:
sent = {"ask_id": int(df_list_all[df_list_all['price'] == df_chance.loc[0]['token.lowest_ask']*10**6]['bigmap_key']), "offer_id": int(df_offer_all[index_offer]['bigmap_key']), "token_id": int(df_offer_all[index_offer]['token.token_id'])}


In [ ]:
df_list_all[df_list_all['price'] == df_chance.loc[0]['token.lowest_ask']*10**6]['bigmap_key']

Series([], Name: bigmap_key, dtype: float64)

In [ ]:
sent = {"ask_id": int(df_list_all[df_list_all['price'] == df_chance.loc[l]['token.lowest_ask']*10**6]['bigmap_key']), "offer_id": int(df_offer_all[index_offer]['bigmap_key']), "token_id": int(df_offer_all[index_offer]['token.token_id'])}

In [ ]:
df_chance

,timestamp,marketplace_event_type,creator_address,token_pk,price,ophash,token.fa_contract,token.highest_offer,token.lowest_ask,token.token_id,token.royalties,royalties
0,2023-02-04T15:53:14+00:00,list_create,tz1eK6niFEPoaa3EfBfzLLXV2A6KVBma3YNs,48655176,-0.000001,onutoWDs4LmaHNBtcFRRPBVQN4B4bVDeVinaRGG2PRi7n6...,KT1CVFAfCjN8hcFJVybwtBZaXb2MDwPHXrpB,11.11,1111.0,0,"[{'decimals': 4, 'amount': 1100}]",0.11


In [ ]:
df_list_all['price']

0    700000.0
Name: price, dtype: float64

In [ ]:
index_list

[False]

In [ ]:
index_offer

[True]

In [ ]:
print(query)

 
query MyQuery {
  offer(
    where: {ophash: {_eq: "ooyRRveJ32NwBPVThfgkF1pSS2wFGK6pKceS7Li8RHborYnTHgX"},token_pk:{_eq: "48574199"}, status: {_eq: "active"}}
    limit: 10
    order_by: {price: desc}
  ) {
    id
    pricen    token_pk
    buyer_address
    status
    timestamp
    bigmap_key
    ophash
    fa_contract
    token {
      token_id
    }
  }
  listing(
    limit: 10
    where: {status: {_eq: "active"}, fa_contract: {_eq: "KT1JVYRDSSosBhYcJvwMEdBUsNkzVNChWnkw "}, token: {token_id: {_eq: "21"}}}
    order_by: {price: asc}
  ) {
    token_pk
    price
    ophash
    seller_address  
    fa_contract
    status
    timestamp
    token {
      token_id
    }
    bigmap_key
  }
}




True

In [ ]:
df.to_json('chance')

In [ ]:
root = tkinter.Tk()

# root window title and dimension
root.title("The run is ended")
root.geometry('500x300')

# Create a messagebox showinfo
#def onClick():
    #tkinter.messagebox.showinfo("Welcome to GFG.", "Hi I'm your message")

# Create a Button
button = Button(root, text="Good to know", command=onClick, height=5, width=10)

# Set the position of button on the top of window.
button.pack(side='bottom')
root.mainloop()

In [ ]:
times = (datetime.now()- timedelta(days=1)).isoformat(timespec='seconds')
query = ' query MyQuery {\n  event(\n    limit: 200\n        where: {marketplace_event_type: {_in: ["list_create", "offer_create", "offer_floor_create"]}, timestamp: {_gt: "2023-01-17T20:17:43"}, token: {highest_offer: {_is_null: false}, lowest_ask: {_is_null: false}, last_metadata_update: {_lt: "'+ f'{times}' + '"}}}\n\n    order_by: {timestamp: desc, token_pk: asc}\n  ) {\n    timestamp\n    marketplace_event_type\n      creator_address\n    token_pk\n    price\n    token {\n      fa_contract\n      highest_offer\n      lowest_ask\n      token_id\n      royalties {\n        decimals\n        amount\n      }\n    }\n    ophash\n  }\n}\n\n '


In [ ]:
print(query)

 query MyQuery {
  event(
    limit: 200
        where: {marketplace_event_type: {_in: ["list_create", "offer_create", "offer_floor_create"]}, timestamp: {_gt: "2023-01-17T20:17:43"}, token: {highest_offer: {_is_null: false}, lowest_ask: {_is_null: false}, last_metadata_update: {_lt: "2023-01-21T08:48:27"}}}

    order_by: {timestamp: desc, token_pk: asc}
  ) {
    timestamp
    marketplace_event_type
      creator_address
    token_pk
    price
    token {
      fa_contract
      highest_offer
      lowest_ask
      token_id
      royalties {
        decimals
        amount
      }
    }
    ophash
  }
}

 


In [ ]:
query = """ 
 query MyQuery {
  event(
    limit: 200
        where: {marketplace_event_type: {_in: ["list_create", "offer_create", "offer_floor_create"]}, timestamp: {_gt: "2023-01-17T20:17:43"}, token: {highest_offer: {_is_null: false}, lowest_ask: {_is_null: false}}}

    order_by: {timestamp: desc, token_pk: asc}
  ) {
    timestamp
    marketplace_event_type
      creator_address
    token_pk
    price
    token {
      fa_contract
      highest_offer
      lowest_ask
      token_id
      royalties {
        decimals
        amount
      }
    }
    ophash
  }
}
"""